# Argumentation framework

### Unit testing

In [ ]:
from utils.iccma.index import get_graphs
from utils.test.index import test

TESTING = False

if TESTING:
    graphs = get_graphs([
        ("scc", "small"),
        ("gr", "small"),
        ("st", "small"),
    ])
    # Complete, stable and preferred
    test(
        iccma_graphs=graphs,
        solvers=["minisat", "manysat", "maplesat", "glucose"],
        semantics=["CO", "ST", "PR"],
        tasks=["SE", "EE"],
        timeout=100,  # seconds
    )
    # Grounded (must not set the solvers)
    test(
        iccma_graphs=graphs,
        solvers=[],
        semantics=["GR"],
        tasks=["SE", "EE"],
        timeout=1,  # seconds
    )
else:
    print("Skipped testing")

### Benchmarking

In [1]:
from utils.benchmark.line_graph import bench
from utils.problem import Problem, Task, Semantics
from typing import List
from graphs import graphs
from itertools import product


TASKS: List[Task] = ["SE", "EE", "DC", "DS"]
SEMANTICS: List[Semantics] = ["CO", "ST", "PR"]
SOLVERS = [
    "minisat",
    "manysat",
    "glucose",
    "maplesat",
]

name, graph = list(graphs.items())[0]
for task, sem in product(TASKS, SEMANTICS):
    problem = Problem(task, sem)
    print(f"{name}: {problem}")
    bench(
        graph["x_label"],
        graph["inputs"],
        graph["gen"],
        SOLVERS,
        problem,
        name,
    )


Initialised _ReproducibleRandom with seed 164925917376
complete: SE-CO
Generation: 100%          
Execution: 100%          
